---
title: "NLP: Data Mining Intro"
author: "Chris Kelly"
date: '02-24-24'
categories: [NLP, TF-IDF, cosine similarity, LSA, topic extraction]
format:
  html:
    code-fold: true
    toc: true
execute: 
  enabled: true
draft: true
---

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

:::{.callout-tip}
Introducing TF-IDF, cosine-similarity and Latent Semantic Analysis
:::

# Quick introduction:

Algorithms like dealing with numbers - they like structure, for input data to be *tidy*. So how can an algorithm start to process unstructured text? And even then - start to extract anything meaningful from it?

<img src="https://media.giphy.com/media/eMBKXi56D0EXC/giphy.gif">

### Some definitions/background

A "token" is a small piece of text. When text data is provided it is split into smaller consistuent pieces (e.g. a sentence is split into words, usually after stemming and stop-word removal). In our example, the tokenisation step involves one-hot encoding each unique word.

A "document" is a collection of tokens associated with a particular sample. In our example, each document will be a restaurant menu. 

An "embedding" is an attempt to create numerical representation of that document in a vector. In our example, we hope that the most important tokens will have the largest values.

This can be useful for similarity search as we will see later (e.g. this user liked this restaurant, so let's recommend them a restaurant with a similar vector).

Finally, the "corpus" is a collection of all documents that the model can learn from. In our example, it the entire colletion of menus.

# Text-mining with TF-IDF:

### What is it used for?

Algorithms like dealing with numbers - they like data to be structured and numerical. So how can an algorithm start to process unstructured text? And even then - start to extract anything meaningful from it?

Text-frequency Inverse-Document-Frequency (TF-IDF) is a technique to find the most important tokens in a document. It is thus a form of information retrieval/text-mining.

<!-- It is good at determining 'global' statistics. By this, we mean it contrasts  the frequency of tokens in a document vs their prevalence across the entire corpus. It does not capture more detailed semantic relationships, particularly since it doesn't care about the ordering of words in a document. -->

<!-- Note further that it is heuristic-based. This means that although the TF-IDF -->

### What is the intuition?

TF-IDF is based on the intuition that tokens that appear more frequently, especially those that are rarer across the entire corpus, are the most important ones relating to that document.

For example, let's contrast two takeaway menus:

* Chicken Kurma Masala, Chicken Tikka Masala, Chicken Curry, Saag Aloo, Pilau Rice, Plain Rice
* Chicken Chow Mein Noodles, Crispy Beef, Chicken and Cashew nut curry, Prawn Crackers, Egg Fried Rice, Steamed rice, Vegetable Noodles

<img src="https://media.giphy.com/media/12xu9HYTRo4Eg0/giphy.gif">

The word 'Chicken' appears the most in the first menu, followed by 'masasla' and 'Rice'. This is "text frequency".

But that's only half the story - since chicken and rice are quite common words in general. So we need a way to measure if a word is rare. The word 'masala' only appears in the first menu, whereas 'chicken' and 'rice' appears in both orders. Hence, the rareness of a word can be determined by how infrequently it appears across all the documents in the corpus. This is "inverse document frequency".

Combining text-frequency and inverse-document frequency scores for each token give it a TF-IDF score. This way, we can derive that the word 'masala' is the most important word from the first order, since it has both high TF and IDF scores.

::: {.column-margin}
TF-IDF can also be done for words, for example splitting the word `manner` into character tokens:

* `n` appears twice in the word
* `m` is rarer

Imagine now that three of the letters are dropped. We are much more likely to guess that the word `m_nn_` could be manner, whereas seeing `_a__er` is far less informative.
:::

#### Text Frequency

<img src="https://media.giphy.com/media/B8Bp8MfpmKbWU/giphy.gif">

We might think that **words that are repeated many times in the menu are more characteristic of that restaurant**

So let's count them:

In [2]:
menus = [
    'Chicken Kurma Masala, Chicken Tikka Masala, Chicken Curry, Pilau Rice, Plain Rice', 
    'Chicken Chow Mein Noodles, Crispy Beef, Chicken curry, Egg Fried Rice, Plain rice, Vegetable Noodles'
    ]

cnt_vec = CountVectorizer()
count_mat = cnt_vec.fit_transform(menus)
pd.DataFrame(
    data = count_mat.todense(), 
    index = ['Order_1', 'Order_2'], 
    columns = cnt_vec.get_feature_names_out()
    )

,beef,chicken,chow,crispy,curry,egg,fried,kurma,masala,mein,noodles,pilau,plain,rice,tikka,vegetable
Order_1,0,3,0,0,1,0,0,1,2,0,0,1,1,2,1,0
Order_2,1,2,1,1,1,1,1,0,0,1,2,0,1,2,0,1


The most common token in the first document is `chicken`, followed by `masala` and `rice`. The second menu also has `chicken` as its most common word, followed `noodles` and `rice`.

In practice, the counts are usually logged. This is a heuristic for diminishing returns - we expect the additional marginal importance for having a token appear once vs twice is greater than it appearing nine times vs 10 times. This is particularly important for longer documents with many tokens.

$$
TF = \log(1 + \text{\# tokens in document})
$$

(we add one as $log(0)$ is undefined)

In [3]:
pd.DataFrame(
    data = np.log(count_mat.todense()+1), 
    index = ['menu_1','menu_2'],
    columns = cnt_vec.get_feature_names_out()
    ).round(2)

,beef,chicken,chow,crispy,curry,egg,fried,kurma,masala,mein,noodles,pilau,plain,rice,tikka,vegetable
menu_1,0.00,1.39,0.00,0.00,0.69,0.00,0.00,0.69,1.1,0.00,0.0,0.69,0.69,1.1,0.69,0.00
menu_2,0.69,1.10,0.69,0.69,0.69,0.69,0.69,0.00,0.0,0.69,1.1,0.00,0.69,1.1,0.00,0.69


#### Inverse Document Frequency

Text-frequency on its own is insufficient. For example in the first order, chicken in general is a common word, so it appearing frequently in a document doesn't provide much information. Masala on the other hand is a rare word, so if it appears frequently this is very informative. 

We thus need to **introduce an additional concept of word uniqueness**, or equivalently the opposite of how frequently it appears across our entire 'corpus' of menu orders.

<img src="https://media.giphy.com/media/5vR6pNsjhoKwo/giphy.gif">

Enter Karen Spärck Jones, with the concept of ‘inverse document frequency’. This is simply the idea that it is not just how often a word appears, **but how unique the word is across all sentences (or ‘documents’), that determines how important it is.**

For example, a rare word like `Masala` only appears in a few documents, so it will have a low document frequency, and thus a high inverse document frequency score.

We often calculate inverse document frequency using the following logic:

$$
\text{idf} = 1+\ln \left(\frac{1+\text{\# docs in corpus}}{1+\text{\# docs token appears in}} \right)
$$

The word chicken appears in both docs, so the idf score is $1+\ln(\frac{1+2}{1+2})=1$. 
On the other hand, the word masala only appears in one doc, so it gets an IDF score of  $1+\ln(\frac{1+2}{1+1})\sim1.4$

In [4]:
idf_vec = TfidfTransformer(smooth_idf=True,norm=None)
idf_mat = idf_vec.fit_transform((count_mat > 0).astype(int))
pd.DataFrame(
    data = idf_mat.todense(),
    index = ['menu_1', 'menu_2'], 
    columns = cnt_vec.get_feature_names_out()
).round(1)

,beef,chicken,chow,crispy,curry,egg,fried,kurma,masala,mein,noodles,pilau,plain,rice,tikka,vegetable
menu_1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.4,1.4,0.0,0.0,1.4,1.0,1.0,1.4,0.0
menu_2,1.4,1.0,1.4,1.4,1.0,1.4,1.4,0.0,0.0,1.4,1.4,0.0,1.0,1.0,0.0,1.4


### TF x IDF

And to combine - TF-IDF is simply the multiplication between the TF and IDF scores, which "combines" the text-frequency and inverse-document-frequency concepts in the same token.

Note that in practice, we do two 

In [5]:
tfidf_vec = TfidfVectorizer(sublinear_tf=True)
tfidf_mat = tfidf_vec.fit_transform(menus)
pd.DataFrame(
    data = tfidf_mat.todense(), 
    index = ['menu_1', 'menu_2'], 
    columns = tfidf_vec.get_feature_names_out()
    ).round(2)

,beef,chicken,chow,crispy,curry,egg,fried,kurma,masala,mein,noodles,pilau,plain,rice,tikka,vegetable
menu_1,0.00,0.46,0.00,0.00,0.22,0.00,0.00,0.31,0.52,0.00,0.00,0.31,0.22,0.37,0.31,0.00
menu_2,0.27,0.32,0.27,0.27,0.19,0.27,0.27,0.00,0.00,0.27,0.46,0.00,0.19,0.32,0.00,0.27


Cool, so combining text frequency and inverse document frequency now reveals the most important word in the first menu is `masala`, and in the second one it is `noodles`. Nice.

Finally, we don't have to limit ourselves to sentences, and can split stings into character tokens, applying the same logic: for the word $\text{queen}$, we would find the letters $q$ and $e$ to be the most informative because of their rarity (IDF) and being repeated (TF). This could be useful in predicting the word being types or correcting mispelling - let's jump into a character-level example when discussing cosine similarity.

# Cosine similarlity

### What is it used for?

Cosine similarity can help measure how similar two words or documents are. For example, we could better match a search to a result using this, whereas 'keywords' would just weight every word equally. 

<img src="https://media.giphy.com/media/13cgadB959Y0BW/giphy.gif">

### What is the intuition?

TF-IDF creates a row of scores for each token in the text, for example the first order had high TF-IDF scores for masala, chicken and rice, and low scores for noodles and fried. If we have another document that has high TF-IF scores for masala, chicken and rice, and low scores for noodles and fried, we might think it is similar to the first order. Cosine similarity gives a measure between one and zero as to how similar the two texts are.

#### Vectorization

To take this further, what we have done using TF-IDF is a form of 'vectorization'. If we were to plot the first order in 16 dimensional space, with one axis for each word, the line remains at zero for the 'beef' axis, travels 0.46 along the 'chicken' axis, etc.

**We can then measure the angle (the cosine) between these lines to get an idea of how similar two documents are.** Cosine similarity is often used to match, how example, a search string to a result. 

This can feel a bit abstract, but hang on in there because this is a key part of intution!

Let's dive into this with an example we can visualise in 3 dimensional space to make this clearer.